<a href="https://colab.research.google.com/github/CelioSGA/LBS_TRACKER/blob/main/LBS_Tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install -q -U google-generativeai
!pip install --upgrade generativeai

In [11]:
import pathlib
import textwrap

from datetime import datetime
import pytz

import google.generativeai as genai

from google.colab import userdata
API_KEY=userdata.get('API_KEY')

from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

genai.configure(api_key=API_KEY)

#estudar melhor
generation_config = {
    "candidate_count": 1,
    "temperature" : 0.7,
    "top_k" : 40,
    "top_p" : 0.95
}

#estudar melhor
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "HATE" : "BLOCK_NONE",
    "DANGEROUS" : "BLOCK_NONE"
}
model = genai.GenerativeModel(model_name = "gemini-1.5-pro-latest",
                              generation_config = generation_config,
                              safety_settings=safety_settings)

#iniciar modelo de acordo com aplicação este é o modelo Alfred

now = datetime.now(pytz.timezone('America/Sao_Paulo'))
local_time = now.strftime('Hoje é %A, %d/%m/%Y, são %H:%M')

print(local_time)

start_model = '''
Este modelo de AI destina-se a rastrear objetos, cargas, veículos entre outros através de torres celulares (LBS) e MAC (WiFi)
Informme as coordenadas através dos dados abaixo e crie um link para o Google maps com a latitude e longitude informados.
Não é necessário que a localização seja precisa, km de desvio são toleráveis.
O dispositivo não possui GPS, apenas modem GSM/LTE e WiFi.
Utilize qualquer base de dados e faça seu melhor esforço.

os dados LBS estão no formato CSV
<start>
cell,bcch,rssi,bsic,cellid,mcc,mnc,lac
1,0240,21,42,5AE5,724,05,1A9B
2,0662,09,38,2E58,724,05,1A9B
3,0251,09,08,5AE3,724,05,1A9B
4,0586,08,00,2DF4,724,05,1A9B
<end>

''' + local_time
Exemplo = '''
{
  "MCC": 310,
  "MNC": 410,
  "LAC": 1234,
  "Cell ID": 5678,
  "Signal Strength": -80
}
'''

#Mudar zona/hora UTC do usuário //Hoje é 6a feira, 10 de maio de 2024, 18:40,

history = [] #carregar history do usuário (Gdrive ou BD) (LIMITE?)

chat = model.start_chat(history = history)
response = chat.send_message(start_model);
print("Traker: ", response.text,"\n")

prompt = ""
while prompt != "fim" :
  prompt = input("Prompt: ")

  if prompt == "fim": break
  if prompt == "end": break
  if prompt == "exit": break

  response = chat.send_message(prompt)
  print("Tracker: ", response.text,"\n")

print(chat.history) #exibe hitórico para análise // salvar em BD/Drive?



Hoje é Saturday, 11/05/2024, são 18:58
Traker:  Infelizmente, determinar a localização apenas com os dados fornecidos é uma tarefa complexa e com alto grau de imprecisão. Isso ocorre porque:

1. **Triangulação Celular Limitada:**  A triangulação celular com base em torres exige informações de pelo menos três torres diferentes para uma estimativa razoável. Os dados fornecidos contêm informações de apenas quatro torres, o que pode ser insuficiente para uma triangulação precisa. 
2. **Ausência de Informações Essenciais:** Para uma triangulação mais precisa, seria necessário saber:
    * **Localização das torres:**  As coordenadas geográficas (latitude e longitude) das torres celulares são cruciais. Sem elas, a triangulação se torna praticamente impossível.
    * **Ângulo de chegada do sinal:**  Conhecer o ângulo em que o sinal atinge cada torre celular ajudaria a determinar a direção da fonte do sinal.
3. **Precisão Limitada do RSSI:** O RSSI (Indicador de Intensidade do Sinal Recebido) é